# import the necessary package

In [16]:
import pandas as pd
import pickle
import random
pd.set_option('display.max_columns',60)
%matplotlib inline
import datetime

# load the dataset

**News Articles ** 

In [17]:
newsData = pd.read_pickle('../Data/Feedly_Processed_DF_cleaned.pkl')

# Country Tags 

**Simple demo **

country code names : http://www.geonames.org/countries/

In [18]:
from geotext import GeoText


In [19]:
countryLabels = pd.read_csv('../Data/Labeled_Data/countries.csv')

In [20]:
newsData.head()

,article_id,title,url,feed_label,content,published,summary,article_text,article_keywords,article_text_len,top_lang
10900,eebb9702,"India, World Bank sign financing agreement for...",http://www.abplive.in/business/india-world-ban...,NEWS WB- All Streams,NaN,2017-12-21 09:22:12,"<table border=""0"" cellspacing=""3"" cellpadding=...","New Delhi [India], Dec 20 (ANI): A financing a...","[institutes, india, skill, financing, training...",1031,en
4268,6832ce57,Rs 40000-crore development projects in limbo i...,http://www.moneycontrol.com/news/business/econ...,NEWS AIIB - All Streams,NaN,2017-12-10 09:40:00,"<table border=""0"" cellspacing=""3"" cellpadding=...","Development projects worth more than Rs 40,000...","[development, crore, andhra, eaps, state, proj...",4390,en
1663,30f8f65e,https://www.the-american-interest.com/2018/01/...,https://www.the-american-interest.com/2018/01/...,NEWS AFDB- All Streams,NaN,2018-01-03 12:21:54,"<table border=""0"" cellspacing=""3"" cellpadding=...",Ten Lessons\n\nDevelopment with Chinese Charac...,"[transitions, university, chinese, united, dev...",575,en
3789,5ec16472,$300 Million to Expand and Upgrade Infrastruct...,https://jis.gov.jm/300-million-to-expand-and-u...,NEWS IDB - All Streams,NaN,2018-04-27 16:28:43,"<table border=""0"" cellspacing=""3"" cellpadding=...","Minister of Industry, Commerce, Agriculture an...","[research, development, agriculture, fisheries...",2621,en
6657,989c9942,ADB Provides $346 Million to Upgrade State Hig...,http://www.business-standard.com/article/news-...,NEWS ADB - All Streams,NaN,2017-12-10 01:00:00,"<table border=""0"" cellspacing=""3"" cellpadding=...",The Asian Development Bank's (ADB) Board of Di...,"[highways, improvement, india, state, upgrade,...",2532,en


In [21]:
newsData.reset_index(inplace = True)

In [22]:
from collections import Counter

In [23]:
#iterate through title to look for country, then iterate though text

def get_top_countries(x):
    """
    given the text and title of an article
    parse out the names of the two most frequently mentioned
    countries in the order of frequency
    """
    geo = GeoText(x)
    counts = Counter(geo.countries)
    country_list =  [country.lower() for (country, counts) in counts.most_common(2)]
    return ", ".join([x for x in country_list]) 


In [24]:
#create combined column of article title and text
newsData['title_and_text'] = newsData['title'] + "\n" + newsData['article_text']

In [25]:
newsData['top_two_countries'] = newsData['title_and_text'].apply(lambda x : get_top_countries(x))


In [26]:
newsData['all_detected_countries'] = newsData['title_and_text'].apply(lambda x : get_top_countries(x))

In [28]:
dfshort = newsData[['article_id', 'top_two_countries', 'all_detected_countries']]
dfshort.to_csv('article_country_tags.csv')

In [29]:
dfshort.head()

,article_id,top_two_countries,all_detected_countries
0,eebb9702,india,india
1,6832ce57,india,india
2,30f8f65e,china,china
3,5ec16472,,
4,989c9942,india,india


In [ ]:
#compare with labeled data and look at our accuracy


In [30]:
joined = countryLabels.merge(dfshort, how = 'inner', on = 'article_id')

In [31]:
df_matches = joined[['article_id', 'Country1','top_two_countries', 'all_detected_countries']]

In [32]:
df_matches.head()

,article_id,Country1,top_two_countries,all_detected_countries
0,10f9ed2,cambodia,"cambodia, japan","cambodia, japan"
1,c0eece9b,afghanistan,"afghanistan, afghanistan","afghanistan, afghanistan"
2,d1d79dd8,bangladesh,"philippines, bangladesh","philippines, bangladesh"
3,f0d65e5,thailand,thailand,thailand
4,4a557358,thailand,"thailand, cambodia","thailand, cambodia"


In [347]:
def get_list(x):
    return x.split(',')

In [392]:
df_matches['country_list'] = df_matches['Country1'].apply(lambda x: get_list(x))

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [393]:
df_matches['top_two_list'] = df_matches['top_two_countries'].apply(lambda x: get_list(x))

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [394]:
predictions = list(df_matches['top_two_list'])

In [395]:
actual= list(df_matches['country_list'])

In [ ]:
# not handling if there are ties of two contries having the same counts...
# check against labelled data

In [327]:
import numpy as np
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)
def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [396]:
mapk(actual, predictions, k = 4)

0.7597619047619049

In [409]:
newsData.head()

,article_id,title,url,feed_label,content,published,summary,article_text,article_keywords,article_text_len,top_lang
10900,eebb9702,"India, World Bank sign financing agreement for...",http://www.abplive.in/business/india-world-ban...,NEWS WB- All Streams,NaN,2017-12-21 09:22:12,"<table border=""0"" cellspacing=""3"" cellpadding=...","New Delhi [India], Dec 20 (ANI): A financing a...","[institutes, india, skill, financing, training...",1031,en
4268,6832ce57,Rs 40000-crore development projects in limbo i...,http://www.moneycontrol.com/news/business/econ...,NEWS AIIB - All Streams,NaN,2017-12-10 09:40:00,"<table border=""0"" cellspacing=""3"" cellpadding=...","Development projects worth more than Rs 40,000...","[development, crore, andhra, eaps, state, proj...",4390,en
1663,30f8f65e,https://www.the-american-interest.com/2018/01/...,https://www.the-american-interest.com/2018/01/...,NEWS AFDB- All Streams,NaN,2018-01-03 12:21:54,"<table border=""0"" cellspacing=""3"" cellpadding=...",Ten Lessons\n\nDevelopment with Chinese Charac...,"[transitions, university, chinese, united, dev...",575,en
3789,5ec16472,$300 Million to Expand and Upgrade Infrastruct...,https://jis.gov.jm/300-million-to-expand-and-u...,NEWS IDB - All Streams,NaN,2018-04-27 16:28:43,"<table border=""0"" cellspacing=""3"" cellpadding=...","Minister of Industry, Commerce, Agriculture an...","[research, development, agriculture, fisheries...",2621,en
6657,989c9942,ADB Provides $346 Million to Upgrade State Hig...,http://www.business-standard.com/article/news-...,NEWS ADB - All Streams,NaN,2017-12-10 01:00:00,"<table border=""0"" cellspacing=""3"" cellpadding=...",The Asian Development Bank's (ADB) Board of Di...,"[highways, improvement, india, state, upgrade,...",2532,en


## parsing out dates in the article text

In [3]:
import datefinder
import datetime

In [7]:
def extract_dates(text:str, base_date:datetime, min_date=datetime.date(2000,1,1),num_dates=5):
    """
    Given a text, parse all dates found in the content
    
    text: article content
    min_date: minimum date for date to be considered valid
    """
    matches = datefinder.find_dates(text,base_date=base_date)
    valid_found_dates = set()
    for match_date in matches:
        match_date = match_date.date()
        if match_date > min_date:
            valid_found_dates.add(match_date)

    return list(valid_found_dates)[:num_dates]

In [ ]:
newsData.head().apply(lambda x : extract_dates (x['article_text'], x['published']), axis=1)

In [ ]:
newsDatanewsData['dates']=newsData.apply(lambda x : extract_dates (x['article_text'], x['published']), axis=1)

In [ ]:
newsData[['article_id','dates']].to_csv('article_date.csv')

## parsing out bank information

In [3]:
bankLabel = pd.read_csv('../Data/Labeled_Data/banks.csv')

In [4]:
bankLabel.head()

,article_id,published,title,url,feed_label,Bank1,Bank2
0,10f9ed2,2018-01-11,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,adb,NaN
1,c0eece9b,2018-05-13,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,adb,NaN
2,d1d79dd8,2018-02-20,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,adb,NaN
3,f0d65e5,2018-02-25,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,adb,NaN
4,4a557358,2018-02-26,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,adb,NaN


In [5]:
bankNewsData = pd.merge(bankLabel,newsData[['article_id','article_text','article_keywords']], on = 'article_id')

In [6]:
bankNames = bankNewsData['Bank1'].unique()
bankNames

array(['adb', 'afdb', 'aiib', 'ebrd', 'eib', 'idb', 'wb'], dtype=object)

In [14]:
def bank_tagging(data, bank_names):
    results =[]
    for i in data.index:
        banks =[]
        for bank in bank_names:
            if bank in data.loc[i, 'feed_label'].replace('-', '').lower().split():
                banks.append(bank)
            else: pass
        results.append([data.loc[i, 'article_id'], ",".join(banks)])
    results_df = pd.DataFrame(results,columns=['article_id', bank])
    return results_df

In [15]:
bank_tagging(newsData, bankNames)

,article_id,article_text,article_keywords,feed_label,url,bank
0,eebb9702,"New Delhi [India], Dec 20 (ANI): A financing a...","[institutes, india, skill, financing, training...",NEWS WB- All Streams,http://www.abplive.in/business/india-world-ban...,['wb']
1,6832ce57,"Development projects worth more than Rs 40,000...","[development, crore, andhra, eaps, state, proj...",NEWS AIIB - All Streams,http://www.moneycontrol.com/news/business/econ...,['aiib']
2,30f8f65e,Ten Lessons\n\nDevelopment with Chinese Charac...,"[transitions, university, chinese, united, dev...",NEWS AFDB- All Streams,https://www.the-american-interest.com/2018/01/...,['afdb']
3,5ec16472,"Minister of Industry, Commerce, Agriculture an...","[research, development, agriculture, fisheries...",NEWS IDB - All Streams,https://jis.gov.jm/300-million-to-expand-and-u...,['idb']
4,989c9942,The Asian Development Bank's (ADB) Board of Di...,"[highways, improvement, india, state, upgrade,...",NEWS ADB - All Streams,http://www.business-standard.com/article/news-...,['adb']
5,67c8512,"by Naim-Ul-Karim\n\nDHAKA, March 2 (Xinhua) --...","[openingup, benefit, party, global, china, chi...",NEWS AIIB - All Streams,http://www.xinhuanet.com/english/2018-03/02/c_...,['aiib']
6,2a5eb487,\n\n\n\nHaiti - Economy : Fruitful meeting bet...,"[meeting, european, president, fruitful, pim, ...",NEWS EIB - All streams,http://www.haitilibre.com/en/news-22990-haiti-...,['eib']
7,a034930a,Highlighting the renewable energy has now beco...,"[150, ireda, inks, line, renewable, loan, cred...",NEWS EIB - All streams,http://www.financialexpress.com/industry/ireda...,['eib']
8,5353b43c,The African Development Bank has approved a $3...,"[green, development, african, sector, renewabl...",NEWS AFDB- All Streams,https://www.cpexecutive.com/post/african-devel...,['afdb']
9,3bdc3c9e,"In 2016, Venezuelan President Nicolás Maduro d...","[bolívar, arc, indigenous, gold, sweeps, minin...",NEWS - Mongabay,https://news.mongabay.com/2018/01/venezuelas-m...,[]
